# GB descriptor

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)


data=pd.read_excel('')
data

X=data.iloc[:,1:-1]
X

Y=data.iloc[:,-1]
Y


from sklearn.model_selection import train_test_split
x_train,x_test_validate,y_train,y_test_validate = train_test_split(X,Y,test_size = 0.4,random_state = 14)
x_validate,x_test,y_validate,y_test = train_test_split(x_test_validate,y_test_validate,test_size = 0.5,random_state = 80)

from sklearn.ensemble import GradientBoostingClassifier


RF = GradientBoostingClassifier(
    #n_estimators=25,
      
    random_state=0
)
RF.fit(x_train,y_train)
y_pred = RF.predict(x_test)



train_score = RF.score(x_train,y_train)
test_score = RF.score(x_test,y_test)
validate_score = RF.score(x_validate,y_validate)
print("train score:{};test score:{};validate_score{}".format(train_score,test_score,validate_score))



from sklearn.metrics import confusion_matrix
C2 = confusion_matrix(y_test,y_pred,labels = [0,1])
print(C2)

import seaborn as sns
sns.set(font_scale = 1.01)
f,ax = plt.subplots()
sns.heatmap(C2,cmap='YlGnBu',annot = True)

ax.set_title('confusion_matrix')
ax.set_xlabel('predict')
ax.set_ylabel('true')
ax_path="confusion_matrix.jpg"; 
heatmap = ax.get_figure()
heatmap.savefig(ax_path, dpi = 600)



from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score


print('Macro accuracy', accuracy_score(y_test, y_pred))
print('Macro precision', precision_score(y_test, y_pred))
print('Macro recall', recall_score(y_test, y_pred))
print('Macro f1-score', f1_score(y_test, y_pred))

from sklearn import metrics
from sklearn.metrics import auc


act = np.array(y_test)
pre = np.array(y_pred)
FPR, TPR, thresholds = metrics.roc_curve(act, pre)
AUC = auc(FPR, TPR)
print('AUC:',AUC)

y_p = RF.predict(x_validate)

C3 = confusion_matrix(y_validate,y_p,labels = [0,1])
print(C3)

import seaborn as sns
import matplotlib.pyplot as plt
sns.set(font_scale = 1.01)
f,ax = plt.subplots()
sns.heatmap(C3,cmap='Greens',annot = True)

ax.set_title('confusion_matrix')
ax.set_xlabel('predict')
ax.set_ylabel('true')
ax_path="confusion_matrix2.jpg"; 
heatmap = ax.get_figure()
heatmap.savefig(ax_path, dpi = 600) 

print('Macro accuracy', accuracy_score(y_validate,y_p))
print('Macro precision', precision_score(y_validate,y_p))
print('Macro recall', recall_score(y_validate,y_p))
print('Macro f1-score', f1_score(y_validate,y_p))

from sklearn import metrics
from sklearn.metrics import auc


act = np.array(y_validate)
pre = np.array(y_p)
FPR, TPR, thresholds = metrics.roc_curve(act, pre)
AUC = auc(FPR, TPR)
print('AUC:',AUC)

new_data = pd.read_excel('yuce2d.xlsx')
x_new=new_data.iloc[:,1:]
x_new

y_new = RF.predict(x_new)
y_new



%matplotlib inline
import shap
shap.initjs()

explainer = shap.TreeExplainer(RF, data=x_train)
shap_values = explainer.shap_values(x_train)

shap.summary_plot(shap_values=shap_values,
                  features=x_train,
                  show = False, plot_type='violin'
                 )
plt.savefig("index3.jpg", dpi = 300)


# GB fingerprint

In [ ]:
import urllib.request
import numpy as np
import pandas as pd
from rdkit import rdBase, Chem, DataStructs 
from rdkit.Avalon import pyAvalonTools
from rdkit.Chem import AllChem
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit.Chem.rdMolDescriptors import GetMorganFingerprint
from rdkit.Chem.AtomPairs import Pairs, Torsions
from rdkit.Chem import Draw
from rdkit.Chem import MACCSkeys
from rdkit.Chem import PandasTools
from rdkit.Chem import Descriptors
from sklearn.cluster import AgglomerativeClustering
import warnings
warnings.filterwarnings("ignore")
from rdkit.Chem.Draw import SimilarityMaps
%matplotlib inline

df = pd.read_excel('') 
df

y=df.iloc[0:84,-1]
y

suppl=PandasTools.AddMoleculeColumnToFrame(df,'smiles')
suppl

morgan_fp = [AllChem.GetMorganFingerprintAsBitVect(mol,2,2048) for mol in df.ROMol]
len(morgan_fp)

mols = [x for x in df.ROMol if x is not None]


morgan_default = AllChem.GetMorganFingerprint(mols[0], 2)
morgan_bitvect = AllChem.GetMorganFingerprintAsBitVect(mols[0], 2, 2048)
print(morgan_default.GetLength(), morgan_bitvect.GetNumBits())



mols=[]
for i in df['smiles']:
    mols.append(Chem.MolFromSmiles(i))

def unhash(mols,radius):
    values=[]
    bit_info=[]
    for mol in mols:
        ECFP_bitinfo = {}
        mfp = AllChem.GetMorganFingerprint(mol, radius, bitInfo=ECFP_bitinfo, useFeatures=False)
        values.append(mfp.GetNonzeroElements())
        bit_info.append(ECFP_bitinfo)
    df_mfp = pd.DataFrame.from_dict(values)
    df_mfp.fillna(0., inplace=True)
    return df_mfp,bit_info

unhashed_fp, bit_info=unhash(mols, 2)
unhashed_fp

bit_info[0]

unhashed_fp1=unhashed_fp.iloc[0:84,]
unhashed_fp1

from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier

X_train, x_test_validate,y_train,y_test_validate = train_test_split(unhashed_fp1, y, test_size = 0.4,random_state = 50)
x_validate,X_test,y_validate,y_test = train_test_split(x_test_validate,y_test_validate,test_size = 0.5,random_state = 0)

model = GradientBoostingClassifier(random_state = 0)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)


y_pred

train_score = model.score(X_train,y_train)
test_score = model.score(X_test,y_test)
validate_score = model.score(x_validate,y_validate)
print("train score:{};test score:{};validate_score{}".format(train_score,test_score,validate_score))

from sklearn.metrics import confusion_matrix
C2 = confusion_matrix(y_test,y_pred,labels = [0,1])
print(C2)

import seaborn as sns
import matplotlib.pyplot as plt
sns.set(font_scale = 1.01)
f,ax = plt.subplots()
sns.heatmap(C2,cmap='YlGnBu',annot = True)

ax.set_title('confusion_matrix')
ax.set_xlabel('predict')
ax.set_ylabel('true')
ax_path="confusion_matrix.jpg"; 
heatmap = ax.get_figure()
heatmap.savefig(ax_path, dpi = 600) 

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

print('Macro accuracy', accuracy_score(y_test, y_pred))
print('Macro precision', precision_score(y_test, y_pred))
print('Macro recall', recall_score(y_test, y_pred))
print('Macro f1-score', f1_score(y_test, y_pred))

from sklearn import metrics
from sklearn.metrics import auc


act = np.array(y_test)
pre = np.array(y_pred)
FPR, TPR, thresholds = metrics.roc_curve(act, pre)
AUC = auc(FPR, TPR)
print('AUC:',AUC)

y_p = model.predict(x_validate)

C3 = confusion_matrix(y_validate,y_p,labels = [0,1])
print(C3)

import seaborn as sns
import matplotlib.pyplot as plt
sns.set(font_scale = 1.01)
f,ax = plt.subplots()
sns.heatmap(C3,cmap='Greens',annot = True)

ax.set_title('confusion_matrix')
ax.set_xlabel('predict')
ax.set_ylabel('true')
ax_path="confusion_matrix2.jpg"; 
heatmap = ax.get_figure()
heatmap.savefig(ax_path, dpi = 600) 

print('Macro accuracy', accuracy_score(y_validate,y_p))
print('Macro precision', precision_score(y_validate,y_p))
print('Macro recall', recall_score(y_validate,y_p))
print('Macro f1-score', f1_score(y_validate,y_p))

from sklearn import metrics
from sklearn.metrics import auc


act = np.array(y_validate)
pre = np.array(y_p)
FPR, TPR, thresholds = metrics.roc_curve(act, pre)
AUC = auc(FPR, TPR)
print('AUC:',AUC)

unhashed_fp2=unhashed_fp.iloc[84:94,]
unhashed_fp2



y_new = model.predict(unhashed_fp2)
y_new





%matplotlib inline
import shap
shap.initjs()



explainer = shap.TreeExplainer(model, data=X_train)
shap_values = explainer.shap_values(X_train)

shap.summary_plot(shap_values=shap_values,
                  features=X_train,
                  feature_names=unhashed_fp.columns)

X_train.shape

columns = unhashed_fp.columns.values.squeeze()
columns.shape


shap_values = np.array(shap_values)
shap_values= shap_values.reshape(-1,272) 
shap_values.shape

unhashed_fp.columns

shap_values_df=pd.DataFrame(shap_values, columns=columns)







def get_info(mol_index, bit_ID, bitinfo,shap_info):
    info=bitinfo[mol_index][bit_ID]
    scaled=preprocessing.minmax_scale(shap_info[bit_ID])
    colour_info=[[cm.viridis(scaled[mol_index])]]*len(info)
    atom_info=[]
    for i in info:
        atom_info.append(i[0])
    colour = dict(zip(atom_info, colour_info))
    
    radius_info=[info[0][1]]*len(info)
    radius = dict(zip(atom_info, radius_info))
    
    return colour, radius

def frag_shap(colour, radius, mol):
    d2d = rdMolDraw2D.MolDraw2DSVG(600,280,300,280)
    d2d.DrawMoleculeWithHighlights(mol,"98513984",colour,colour,radius,{})
    d2d.FinishDrawing()
    return SVG(d2d.GetDrawingText())



from sklearn import preprocessing

scaled=preprocessing.minmax_scale(shap_values_df[3218693969])

import matplotlib.cm as cm

import numpy as np
from rdkit import Chem
from IPython import display
import urllib.request 
import pandas as pd
from rdkit.Chem import AllChem, Draw, Descriptors, PandasTools
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
#import lightgbm as lgb
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from sklearn import preprocessing
from rdkit.Chem.Draw import rdMolDraw2D
from IPython.display import SVG

colour, radius = get_info(0, 2246699815, bit_info, shap_values_df)
frag_shap(colour, radius,mols[0])

colour, radius = get_info(18, 3638823234, bit_info, shap_values_df)
frag_shap(colour, radius,mols[18])



# RF descriptor

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)


data=pd.read_excel('fz2d.xlsx')
data

X=data.iloc[:,1:-1]
X

Y=data.iloc[:,-1]
Y



from sklearn.model_selection import train_test_split
x_train,x_test_validate,y_train,y_test_validate = train_test_split(X,Y,test_size = 0.4,random_state = 67)
x_validate,x_test,y_validate,y_test = train_test_split(x_test_validate,y_test_validate,test_size = 0.5,random_state =70)

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(
    #n_estimators=25,
      
    random_state=0
)
RF.fit(x_train,y_train)
y_pred = RF.predict(x_test)



train_score = RF.score(x_train,y_train)
test_score =RF.score(x_test,y_test)
validate_score = RF.score(x_validate,y_validate)
print("train score:{};test score:{};validate_score{}".format(train_score,test_score,validate_score))



from sklearn.metrics import confusion_matrix
C2 = confusion_matrix(y_test,y_pred,labels = [0,1])
print(C2)

import seaborn as sns
sns.set(font_scale = 1.01)
f,ax = plt.subplots()
sns.heatmap(C2,cmap='YlGnBu',annot = True)

ax.set_title('confusion_matrix')
ax.set_xlabel('predict')
ax.set_ylabel('true')
ax_path="confusion_matrix.jpg"; 
heatmap = ax.get_figure()
heatmap.savefig(ax_path, dpi = 600) 

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score




print('Macro accuracy', accuracy_score(y_test, y_pred))
print('Macro precision', precision_score(y_test, y_pred))
print('Macro recall', recall_score(y_test, y_pred))
print('Macro f1-score', f1_score(y_test, y_pred))

from sklearn import metrics
from sklearn.metrics import auc


act = np.array(y_test)
pre = np.array(y_pred)
FPR, TPR, thresholds = metrics.roc_curve(act, pre)
AUC = auc(FPR, TPR)
print('AUC:',AUC)

y_p = RF.predict(x_validate)

C3 = confusion_matrix(y_validate,y_p,labels = [0,1])
print(C3)

import seaborn as sns
import matplotlib.pyplot as plt
sns.set(font_scale = 1.01)
f,ax = plt.subplots()
sns.heatmap(C3,cmap='Greens',annot = True)

ax.set_title('confusion_matrix')
ax.set_xlabel('predict')
ax.set_ylabel('true')
ax_path="confusion_matrix2.jpg"; 
heatmap = ax.get_figure()
heatmap.savefig(ax_path, dpi = 600) 

print('Macro accuracy', accuracy_score(y_validate,y_p))
print('Macro precision', precision_score(y_validate,y_p))
print('Macro recall', recall_score(y_validate,y_p))
print('Macro f1-score', f1_score(y_validate,y_p))

from sklearn import metrics
from sklearn.metrics import auc


act = np.array(y_validate)
pre = np.array(y_p)
FPR, TPR, thresholds = metrics.roc_curve(act, pre)
AUC = auc(FPR, TPR)
print('AUC:',AUC)

new_data = pd.read_excel('yuce2d.xlsx')
x_new=new_data.iloc[:,1:]
x_new

y_new = RF.predict(x_new)
y_new



data = pd.read_excel('fz2d.xlsx')
X_d =data.columns.tolist()[1:]
del X_d[-1]
X_d

from pandas.core.frame import DataFrame
X_dis = DataFrame(X_d)
X_dis

X_display= pd.DataFrame(X_dis.values.T,columns = X_dis.index,index = X_dis.columns)
X_display



matplotlib inline
import shap
shap.initjs()


explainer = shap.TreeExplainer(RF, data=x_train)
shap_values = explainer.shap_values(x_train)

shap.summary_plot(shap_values=shap_values,
                  features=x_train,
                  plot_type="bar",show = False)
plt.savefig("index3.jpg", dpi = 300)



# RF fingerprint

In [ ]:
import urllib.request
import numpy as np
import pandas as pd
from rdkit import rdBase, Chem, DataStructs 
from rdkit.Avalon import pyAvalonTools
from rdkit.Chem import AllChem
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit.Chem.rdMolDescriptors import GetMorganFingerprint
from rdkit.Chem.AtomPairs import Pairs, Torsions
from rdkit.Chem import Draw
from rdkit.Chem import MACCSkeys
from rdkit.Chem import PandasTools
from rdkit.Chem import Descriptors
from sklearn.cluster import AgglomerativeClustering
import warnings
warnings.filterwarnings("ignore")
from rdkit.Chem.Draw import SimilarityMaps
%matplotlib inline

df = pd.read_excel('') 
df

y=df.iloc[0:84,-1]
y

suppl=PandasTools.AddMoleculeColumnToFrame(df,'smiles')
suppl

morgan_fp = [AllChem.GetMorganFingerprintAsBitVect(mol,2,2048) for mol in df.ROMol]
len(morgan_fp)

mols = [x for x in df.ROMol if x is not None]


morgan_default = AllChem.GetMorganFingerprint(mols[0], 2)
morgan_bitvect = AllChem.GetMorganFingerprintAsBitVect(mols[0], 2, 2048)
print(morgan_default.GetLength(), morgan_bitvect.GetNumBits())



mols=[]
for i in df['smiles']:
    mols.append(Chem.MolFromSmiles(i))

def unhash(mols,radius):
    values=[]
    bit_info=[]
    for mol in mols:
        ECFP_bitinfo = {}
        mfp = AllChem.GetMorganFingerprint(mol, radius, bitInfo=ECFP_bitinfo, useFeatures=False)
        values.append(mfp.GetNonzeroElements())
        bit_info.append(ECFP_bitinfo)
    df_mfp = pd.DataFrame.from_dict(values)
    df_mfp.fillna(0., inplace=True)
    return df_mfp,bit_info

unhashed_fp, bit_info=unhash(mols, 2)
unhashed_fp

bit_info[2]

unhashed_fp1=unhashed_fp.iloc[0:84,]
unhashed_fp1




from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

X_train, x_test_validate,y_train,y_test_validate = train_test_split(unhashed_fp1, y, test_size = 0.4,random_state = 8)
x_validate,X_test,y_validate,y_test = train_test_split(x_test_validate,y_test_validate,test_size = 0.5,random_state = 0)


model = RandomForestClassifier(random_state=0)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)


y_pred

X_train







train_score = model.score(X_train,y_train)
test_score = model.score(X_test,y_test)
validate_score = model.score(x_validate,y_validate)
print("train score:{};test score:{};validate_score{}".format(train_score,test_score,validate_score))

from sklearn.metrics import confusion_matrix
C2 = confusion_matrix(y_test,y_pred,labels = [0,1])
print(C2)

import seaborn as sns
import matplotlib.pyplot as plt
sns.set(font_scale = 1.01)
f,ax = plt.subplots()
sns.heatmap(C2,cmap='YlGnBu',annot = True)

ax.set_title('confusion_matrix')
ax.set_xlabel('predict')
ax.set_ylabel('true')
ax_path="confusion_matrix.jpg"; 
heatmap = ax.get_figure()
heatmap.savefig(ax_path, dpi = 600)

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

print('Macro accuracy', accuracy_score(y_test, y_pred))
print('Macro precision', precision_score(y_test, y_pred))
print('Macro recall', recall_score(y_test, y_pred))
print('Macro f1-score', f1_score(y_test, y_pred))

from sklearn import metrics
from sklearn.metrics import auc


act = np.array(y_test)
pre = np.array(y_pred)
FPR, TPR, thresholds = metrics.roc_curve(act, pre)
AUC = auc(FPR, TPR)
print('AUC:',AUC)

y_p = model.predict(x_validate)

C3 = confusion_matrix(y_validate,y_p,labels = [0,1])
print(C3)

import seaborn as sns
import matplotlib.pyplot as plt
sns.set(font_scale = 1.01)
f,ax = plt.subplots()
sns.heatmap(C3,cmap='Greens',annot = True)

ax.set_title('confusion_matrix')
ax.set_xlabel('predict')
ax.set_ylabel('true')
ax_path="confusion_matrix2.jpg"; 
heatmap = ax.get_figure()
heatmap.savefig(ax_path, dpi = 600) 
print('Macro accuracy', accuracy_score(y_validate,y_p))
print('Macro precision', precision_score(y_validate,y_p))
print('Macro recall', recall_score(y_validate,y_p))
print('Macro f1-score', f1_score(y_validate,y_p))

from sklearn import metrics
from sklearn.metrics import auc


act = np.array(y_validate)
pre = np.array(y_p)
FPR, TPR, thresholds = metrics.roc_curve(act, pre)
AUC = auc(FPR, TPR)
print('AUC:',AUC)



unhashed_fp2=unhashed_fp.iloc[84:94,]
unhashed_fp2



y_new = model.predict(unhashed_fp2)
y_new





%matplotlib inline
import shap
shap.initjs()



explainer = shap.TreeExplainer(model, data=X_train)
shap_values = explainer.shap_values(X_train)

shap.summary_plot(shap_values=shap_values,
                  features=X_train,
                  feature_names=unhashed_fp.columns)

X_train.shape

columns = unhashed_fp.columns.values.squeeze()
columns.shape


shap_values = np.array(shap_values)
shap_values= shap_values.reshape(-1,272) 
shap_values.shape

unhashed_fp.columns

shap_values_df=pd.DataFrame(shap_values, columns=columns)





def get_info(mol_index, bit_ID, bitinfo,shap_info):
    info=bitinfo[mol_index][bit_ID]
    scaled=preprocessing.minmax_scale(shap_info[bit_ID])
    colour_info=[[cm.viridis(scaled[mol_index])]]*len(info)
    atom_info=[]
    for i in info:
        atom_info.append(i[0])
    colour = dict(zip(atom_info, colour_info))
    
    radius_info=[info[0][1]]*len(info)
    radius = dict(zip(atom_info, radius_info))
    
    return colour, radius



def frag_shap(colour, radius, mol):
    d2d = rdMolDraw2D.MolDraw2DSVG(600,280,300,280)
    d2d.DrawMoleculeWithHighlights(mol,"2246728737",colour,colour,radius,{})
    d2d.FinishDrawing()
    return SVG(d2d.GetDrawingText())

from sklearn import preprocessing

scaled=preprocessing.minmax_scale(shap_values_df[2246728737])


import matplotlib.cm as cm

import numpy as np
from rdkit import Chem
from IPython import display
import urllib.request 
import pandas as pd
from rdkit.Chem import AllChem, Draw, Descriptors, PandasTools
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
#import lightgbm as lgb
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from sklearn import preprocessing
from rdkit.Chem.Draw import rdMolDraw2D
from IPython.display import SVG

colour, radius = get_info(6,2246728737, bit_info, shap_values_df)
frag_shap(colour, radius,mols[18])









colour, radius = get_info(93,3938119383, bit_info, shap_values_df)
frag_shap(colour, radius,mols[18])

colour, radius = get_info(35,3938119383, bit_info, shap_values_df)
frag_shap(colour, radius,mols[18])

colour, radius = get_info(43,3938119383, bit_info, shap_values_df)
frag_shap(colour, radius,mols[18])

colour, radius = get_info(45,3938119383, bit_info, shap_values_df)
frag_shap(colour, radius,mols[18])

colour, radius = get_info(77,3938119383, bit_info, shap_values_df)
frag_shap(colour, radius,mols[18])



colour, radius = get_info(88,3938119383, bit_info, shap_values_df)
frag_shap(colour, radius,mols[18])





# Stack descriptor

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)


data=pd.read_excel('')
data

X=data.iloc[:,1:-1]
X

Y=data.iloc[:,-1]
Y






from sklearn.model_selection import train_test_split
x_train,x_test_validate,y_train,y_test_validate = train_test_split(X,Y,test_size = 0.4,random_state = 20)
x_validate,x_test,y_validate,y_test = train_test_split(x_test_validate,y_test_validate,test_size = 0.5,random_state = 1)

from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

estimators = [
    ('rf', RandomForestClassifier(n_estimators=10, random_state=42)),
    ('gb', GradientBoostingClassifier(
    n_estimators=25,
      
    random_state=0
))
]
RF = StackingClassifier(
    estimators=estimators, #final_estimator=LogisticRegression()
)
RF.fit(x_train,y_train)
y_pred = RF.predict(x_test)



train_score = RF.score(x_train,y_train)
test_score = RF.score(x_test,y_test)
validate_score = RF.score(x_validate,y_validate)
print("train score:{};test score:{};validate_score{}".format(train_score,test_score,validate_score))



from sklearn.metrics import confusion_matrix
C2 = confusion_matrix(y_test,y_pred,labels = [0,1])
print(C2)

import seaborn as sns
sns.set(font_scale = 1.01)
f,ax = plt.subplots()
sns.heatmap(C2,cmap='Blues',annot = True)

ax.set_title('confusion_matrix')
ax.set_xlabel('predict')
ax.set_ylabel('true')
ax_path="confusion_matrix.jpg"; 
heatmap = ax.get_figure()
heatmap.savefig(ax_path, dpi = 600) 

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

print('Macro accuracy', accuracy_score(y_test, y_pred))
print('Macro precision', precision_score(y_test, y_pred))
print('Macro recall', recall_score(y_test, y_pred))
print('Macro f1-score', f1_score(y_test, y_pred))

from sklearn import metrics
from sklearn.metrics import auc


act = np.array(y_test)
pre = np.array(y_pred)
FPR, TPR, thresholds = metrics.roc_curve(act, pre)
AUC = auc(FPR, TPR)
print('AUC:',AUC)

y_p = RF.predict(x_validate)

C3 = confusion_matrix(y_validate,y_p,labels = [0,1])
print(C3)

import seaborn as sns
import matplotlib.pyplot as plt
sns.set(font_scale = 1.01)
f,ax = plt.subplots()
sns.heatmap(C3,cmap='Greens',annot = True)

ax.set_title('confusion_matrix')
ax.set_xlabel('predict')
ax.set_ylabel('true')
ax_path="confusion_matrix2.jpg"; 
heatmap = ax.get_figure()
heatmap.savefig(ax_path, dpi = 600) 

print('Macro accuracy', accuracy_score(y_validate,y_p))
print('Macro precision', precision_score(y_validate,y_p))
print('Macro recall', recall_score(y_validate,y_p))
print('Macro f1-score', f1_score(y_validate,y_p))

from sklearn import metrics
from sklearn.metrics import auc


act = np.array(y_validate)
pre = np.array(y_p)
FPR, TPR, thresholds = metrics.roc_curve(act, pre)
AUC = auc(FPR, TPR)
print('AUC:',AUC)

new_data = pd.read_excel('yuce2d.xlsx')
x_new=new_data.iloc[:,1:]
x_new

y_new = RF.predict(x_new)
y_new

%matplotlib inline
import shap
shap.initjs()

explainer = shap.KernelExplainer(RF.predict, data=x_train)
shap_values = explainer.shap_values(x_train)

shap.summary_plot(shap_values=shap_values,
                  features=x_train,
                  show = False, plot_type='violin'
                 )
plt.savefig("index3.jpg", dpi = 300)



# stack fingerprint

In [ ]:
import urllib.request
import numpy as np
import pandas as pd
from rdkit import rdBase, Chem, DataStructs 
from rdkit.Avalon import pyAvalonTools
from rdkit.Chem import AllChem
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit.Chem.rdMolDescriptors import GetMorganFingerprint
from rdkit.Chem.AtomPairs import Pairs, Torsions
from rdkit.Chem import Draw
from rdkit.Chem import MACCSkeys
from rdkit.Chem import PandasTools
from rdkit.Chem import Descriptors
from sklearn.cluster import AgglomerativeClustering
import warnings
warnings.filterwarnings("ignore")
from rdkit.Chem.Draw import SimilarityMaps
%matplotlib inline

df = pd.read_excel('') 
df

y=df.iloc[:84,-1]
y

suppl=PandasTools.AddMoleculeColumnToFrame(df,'smiles')
suppl

morgan_fp = [AllChem.GetMorganFingerprintAsBitVect(mol,2,2048) for mol in df.ROMol]
len(morgan_fp)

mols = [x for x in df.ROMol if x is not None]


morgan_default = AllChem.GetMorganFingerprint(mols[0], 2)
morgan_bitvect = AllChem.GetMorganFingerprintAsBitVect(mols[0], 2, 2048)
print(morgan_default.GetLength(), morgan_bitvect.GetNumBits())



mols=[]
for i in df['smiles']:
    mols.append(Chem.MolFromSmiles(i))

def unhash(mols,radius):
    values=[]
    bit_info=[]
    for mol in mols:
        ECFP_bitinfo = {}
        mfp = AllChem.GetMorganFingerprint(mol, radius, bitInfo=ECFP_bitinfo, useFeatures=False)
        values.append(mfp.GetNonzeroElements())
        bit_info.append(ECFP_bitinfo)
    df_mfp = pd.DataFrame.from_dict(values)
    df_mfp.fillna(0., inplace=True)
    return df_mfp,bit_info

unhashed_fp, bit_info=unhash(mols, 2)
unhashed_fp

bit_info[0]

unhashed_fp1=unhashed_fp.iloc[0:84,]
unhashed_fp1



from sklearn.model_selection import train_test_split
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
X_train, x_test_validate,y_train,y_test_validate = train_test_split(unhashed_fp1, y, test_size = 0.4,random_state = 1)
x_validate,X_test,y_validate,y_test = train_test_split(x_test_validate,y_test_validate,test_size = 0.5,random_state = 0)
estimators = [
    ('rf', RandomForestClassifier(n_estimators=20, random_state=10)),
    ('gb', GradientBoostingClassifier(
    n_estimators=5,
      
    random_state=0
))
]

model = StackingClassifier(
    estimators=estimators, 
  #  final_estimator=KNeighborsClassifier()
)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)





y_pred

train_score = model.score(X_train,y_train)
test_score = model.score(X_test,y_test)
validate_score = model.score(x_validate,y_validate)
print("train score:{};test score:{};validate_score{}".format(train_score,test_score,validate_score))

from sklearn.metrics import confusion_matrix
C2 = confusion_matrix(y_test,y_pred,labels = [0,1])
print(C2)

import seaborn as sns
import matplotlib.pyplot as plt
sns.set(font_scale = 1.01)
f,ax = plt.subplots()
sns.heatmap(C2,cmap='Blues',annot = True)

ax.set_title('confusion_matrix')
ax.set_xlabel('predict')
ax.set_ylabel('true')
ax_path="confusion_matrix.jpg"; 
heatmap = ax.get_figure()
heatmap.savefig(ax_path, dpi = 600) 

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score


print('Macro accuracy', accuracy_score(y_test, y_pred))
print('Macro precision', precision_score(y_test, y_pred))
print('Macro recall', recall_score(y_test, y_pred))
print('Macro f1-score', f1_score(y_test, y_pred))

from sklearn import metrics
from sklearn.metrics import auc


act = np.array(y_test)
pre = np.array(y_pred)
FPR, TPR, thresholds = metrics.roc_curve(act, pre)
AUC = auc(FPR, TPR)
print('AUC:',AUC)

y_p = model.predict(x_validate)

C3 = confusion_matrix(y_validate,y_p,labels = [0,1])
print(C3)

import seaborn as sns
import matplotlib.pyplot as plt
sns.set(font_scale = 1.01)
f,ax = plt.subplots()
sns.heatmap(C3,cmap='Greens',annot = True)

ax.set_title('confusion_matrix')
ax.set_xlabel('predict')
ax.set_ylabel('true')
ax_path="confusion_matrix2.jpg"; 
heatmap = ax.get_figure()
heatmap.savefig(ax_path, dpi = 600) 

print('Macro accuracy', accuracy_score(y_validate,y_p))
print('Macro precision', precision_score(y_validate,y_p))
print('Macro recall', recall_score(y_validate,y_p))
print('Macro f1-score', f1_score(y_validate,y_p))

from sklearn import metrics
from sklearn.metrics import auc


act = np.array(y_validate)
pre = np.array(y_p)
FPR, TPR, thresholds = metrics.roc_curve(act, pre)
AUC = auc(FPR, TPR)
print('AUC:',AUC)





unhashed_fp2=unhashed_fp.iloc[84:94,]
unhashed_fp2



y_new = model.predict(unhashed_fp2)
y_new



from sklearn.model_selection import cross_val_score
#scoring = ['precision_macro', 'recall_macro']



scores = cross_val_score(model,unhashed_fp1, y, cv=20, scoring='average_precision')

scores.mean()

scores

def visualize_cv(cv, X, y): 
    fig, ax = plt.subplots(figsize=(10, 5)) 
 
    for ii, (tr, tt) in enumerate(cv.split(X, y)): 
        p1 = ax.scatter(tr, [ii] * len(tr), c="#221f1f", marker="_", lw=10) 
        p2 = ax.scatter(tt, [ii] * len(tt), c="#b20710", marker="_", lw=10) 
        ax.set( 
            title=cv.__class__.__name__, 
            xlabel="Data Index", 
            ylabel="CV Iteration", 
            ylim=[cv.n_splits, -1], 
        ) 
        ax.legend([p1, p2], ["Training", "Validation"]) 
 
    plt.show()


from sklearn.model_selection import KFold 
cv = KFold(n_splits=20)
visualize_cv(cv, unhashed_fp1, y)





scores

from sklearn.model_selection import cross_val_score,StratifiedKFold

stratifiedkf=StratifiedKFold(n_splits=10)


score=cross_val_score(model,unhashed_fp1, y,cv=stratifiedkf)
score


score.mean()

from sklearn.model_selection import LeaveOneOut,cross_val_score



loo=LeaveOneOut()

score=cross_val_score(model,unhashed_fp1, y,cv=loo ,scoring='accuracy')
score

score.mean()





%matplotlib inline
import shap
shap.initjs()




explainer = shap.KernelExplainer(model.predict, data=X_train)
shap_values = explainer.shap_values(X_train)




shap.summary_plot(shap_values=shap_values,
                  features=X_train,
                  feature_names=unhashed_fp.columns)

X_train.shape

columns = unhashed_fp.columns.values.squeeze()
columns.shape



unhashed_fp.columns

shap_values_df=pd.DataFrame(shap_values, columns=columns)







def get_info(mol_index, bit_ID, bitinfo,shap_info):
    info=bitinfo[mol_index][bit_ID]
    scaled=preprocessing.minmax_scale(shap_info[bit_ID])
    colour_info=[[cm.viridis(scaled[mol_index])]]*len(info)
    atom_info=[]
    for i in info:
        atom_info.append(i[0])
    colour = dict(zip(atom_info, colour_info))
    
    radius_info=[info[0][1]]*len(info)
    radius = dict(zip(atom_info, radius_info))
    
    return colour, radius

def frag_shap(colour, radius, mol):
    d2d = rdMolDraw2D.MolDraw2DSVG(600,280,300,280)
    d2d.DrawMoleculeWithHighlights(mol,"98513984",colour,colour,radius,{})
    d2d.FinishDrawing()
    return SVG(d2d.GetDrawingText())



from sklearn import preprocessing

scaled=preprocessing.minmax_scale(shap_values_df[3218693969])

import matplotlib.cm as cm

import numpy as np
from rdkit import Chem
from IPython import display
import urllib.request 
import pandas as pd
from rdkit.Chem import AllChem, Draw, Descriptors, PandasTools
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
#import lightgbm as lgb
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from sklearn import preprocessing
from rdkit.Chem.Draw import rdMolDraw2D
from IPython.display import SVG

colour, radius = get_info(10, 161963127, bit_info, shap_values_df)
frag_shap(colour, radius,mols[10])

colour, radius = get_info(18, 161963127, bit_info, shap_values_df)
frag_shap(colour, radius,mols[18])



# MLP

In [ ]:
import urllib.request
import numpy as np
import pandas as pd
from rdkit import rdBase, Chem, DataStructs 
from rdkit.Avalon import pyAvalonTools
from rdkit.Chem import AllChem
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit.Chem.rdMolDescriptors import GetMorganFingerprint
from rdkit.Chem.AtomPairs import Pairs, Torsions
from rdkit.Chem import Draw
from rdkit.Chem import MACCSkeys
from rdkit.Chem import PandasTools
from rdkit.Chem import Descriptors
from sklearn.cluster import AgglomerativeClustering
import warnings
warnings.filterwarnings("ignore")
from rdkit.Chem.Draw import SimilarityMaps
%matplotlib inline

df = pd.read_excel('fzzw.xlsx') 
df

y=df.iloc[0:92,-1]
y

suppl=PandasTools.AddMoleculeColumnToFrame(df,'smiles')
suppl

morgan_fp = [AllChem.GetMorganFingerprintAsBitVect(mol,2,2048) for mol in df.ROMol]
len(morgan_fp)

mols = [x for x in df.ROMol if x is not None]


morgan_default = AllChem.GetMorganFingerprint(mols[0], 2)
morgan_bitvect = AllChem.GetMorganFingerprintAsBitVect(mols[0], 2, 2048)
print(morgan_default.GetLength(), morgan_bitvect.GetNumBits())



mols=[]
for i in df['smiles']:
    mols.append(Chem.MolFromSmiles(i))

def unhash(mols,radius):
    values=[]
    bit_info=[]
    for mol in mols:
        ECFP_bitinfo = {}
        mfp = AllChem.GetMorganFingerprint(mol, radius, bitInfo=ECFP_bitinfo, useFeatures=False)
        values.append(mfp.GetNonzeroElements())
        bit_info.append(ECFP_bitinfo)
    df_mfp = pd.DataFrame.from_dict(values)
    df_mfp.fillna(0., inplace=True)
    return df_mfp,bit_info

unhashed_fp, bit_info=unhash(mols, 4)
unhashed_fp

bit_info[2]

unhashed_fp1=unhashed_fp.iloc[0:92,]
unhashed_fp1


from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(unhashed_fp1,y,test_size = 0.2,random_state = 0)


from sklearn.neural_network import MLPClassifier


from sklearn.model_selection import cross_val_score, GridSearchCV
mlp_clf__tuned_parameters = {
                             "solver": ['sgd', 'lbfgs','adam'],
                             "max_iter": [15,20,25,50,100,200,600,800],
                             "alpha": [0.0000001,0.000001,0.00001,0.0001,0.001,0.01,0.1],
                             "verbose": [True]
                             }
mlp = MLPClassifier(random_state=42)
estimator=GridSearchCV(mlp,mlp_clf__tuned_parameters,n_jobs=1,cv=5,verbose=5)
estimator.fit(x_train.astype(float), y_train.astype(float))
predi_new=estimator.predict(x_test)



print(estimator .best_params_)

train_score = estimator.score(x_train,y_train)
test_score = estimator.score(x_test,y_test)
validate_score = estimator.score(x_validate,y_validate)
print("train score:{};test score:{};validate_score:{}".format(train_score,test_score,validate_score))


for i in range(0,100,1):
    x_train,x_test,y_train,y_test = train_test_split(unhashed_fp1,y,test_size = 0.2,random_state =4)

    estimator = MLPClassifier(alpha=0.0000001, max_iter=50, solver='lbfgs', verbose= True,random_state=i)
    estimator.fit(x_train, y_train)

    train_score = estimator.score(x_train,y_train)
    test_score = estimator.score(x_test,y_test)

    print(str(i)+'  '+"train score:{};test score:{}".format(train_score,test_score))



x_train,x_test,y_train,y_test = train_test_split(unhashed_fp1,y,test_size = 0.2,random_state =4)


model =  MLPClassifier(alpha=0.0000001, max_iter=50, solver='lbfgs', verbose= True,random_state=98)
model.fit(x_train, y_train)



y_pred=model.predict(x_test)











from sklearn.metrics import confusion_matrix
C2 = confusion_matrix(y_test,y_pred,labels = [0,1])
print(C2)

import seaborn as sns
import matplotlib.pyplot as plt
sns.set(font_scale = 1.01)
f,ax = plt.subplots()
sns.heatmap(C2,cmap='Oranges',annot = True)

ax.set_title('confusion_matrix')
ax.set_xlabel('predict')
ax.set_ylabel('true')
ax_path="confusion_matrix.jpg"; 
heatmap = ax.get_figure()
heatmap.savefig(ax_path, dpi = 600) # dpi是每英寸点数，控制生成图片的清晰度

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

print('Macro accuracy', accuracy_score(y_test, y_pred))
print('Macro precision', precision_score(y_test, y_pred))
print('Macro recall', recall_score(y_test, y_pred))
print('Macro f1-score', f1_score(y_test, y_pred))

from sklearn import metrics
from sklearn.metrics import auc


act = np.array(y_test)
pre = np.array(y_pred)

FPR, TPR, thresholds = metrics.roc_curve(act, pre)
AUC = auc(FPR, TPR)
print('AUC:',AUC)

from sklearn.model_selection import cross_val_score

scores = cross_val_score(model,unhashed_fp1,y, cv=10)
scores

scores.mean()

unhashed_fp2=unhashed_fp.iloc[92:94,]
unhashed_fp2



y_new = model.predict(unhashed_fp2)
y_new





%matplotlib inline
import shap
shap.initjs()



explainer = shap.TreeExplainer(model, data=X_train)
shap_values = explainer.shap_values(X_train)

shap.summary_plot(shap_values=shap_values,
                  features=X_train,
                  feature_names=unhashed_fp.columns)

X_train.shape

columns = unhashed_fp.columns.values.squeeze()
columns.shape


shap_values = np.array(shap_values)
shap_values= shap_values.reshape(-1,272) 
shap_values.shape

unhashed_fp.columns

shap_values_df=pd.DataFrame(shap_values, columns=columns)





def get_info(mol_index, bit_ID, bitinfo,shap_info):
    info=bitinfo[mol_index][bit_ID]
    scaled=preprocessing.minmax_scale(shap_info[bit_ID])
    colour_info=[[cm.viridis(scaled[mol_index])]]*len(info)
    atom_info=[]
    for i in info:
        atom_info.append(i[0])
    colour = dict(zip(atom_info, colour_info))
    
    radius_info=[info[0][1]]*len(info)
    radius = dict(zip(atom_info, radius_info))
    
    return colour, radius



def frag_shap(colour, radius, mol):
    d2d = rdMolDraw2D.MolDraw2DSVG(600,280,300,280)
    d2d.DrawMoleculeWithHighlights(mol,"2246728737",colour,colour,radius,{})
    d2d.FinishDrawing()
    return SVG(d2d.GetDrawingText())

from sklearn import preprocessing

scaled=preprocessing.minmax_scale(shap_values_df[2246728737])


import matplotlib.cm as cm

import numpy as np
from rdkit import Chem
from IPython import display
import urllib.request 
import pandas as pd
from rdkit.Chem import AllChem, Draw, Descriptors, PandasTools
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
#import lightgbm as lgb
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from sklearn import preprocessing
from rdkit.Chem.Draw import rdMolDraw2D
from IPython.display import SVG

colour, radius = get_info(6,2246728737, bit_info, shap_values_df)
frag_shap(colour, radius,mols[18])









colour, radius = get_info(93,3938119383, bit_info, shap_values_df)
frag_shap(colour, radius,mols[18])

colour, radius = get_info(35,3938119383, bit_info, shap_values_df)
frag_shap(colour, radius,mols[18])

colour, radius = get_info(43,3938119383, bit_info, shap_values_df)
frag_shap(colour, radius,mols[18])

colour, radius = get_info(45,3938119383, bit_info, shap_values_df)
frag_shap(colour, radius,mols[18])

colour, radius = get_info(77,3938119383, bit_info, shap_values_df)
frag_shap(colour, radius,mols[18])



colour, radius = get_info(88,3938119383, bit_info, shap_values_df)
frag_shap(colour, radius,mols[18])


